In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras import backend 

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.utils.vis_utils import plot_model

import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
import numpy as np
import mne
import os
import cv2
import random
import math
import shutil

In [2]:
# constants and Hyper-parameters
batch_size = 32
num_channels = 3
num_classes_of_each_cat = 10
num_classes = 4*num_classes_of_each_cat
image_height = 128
image_width = 320
latent_dim = 128

generator_in_channels = latent_dim + num_classes
critic_in_channels = num_channels + num_classes
PARTIAL_IMG_SHAPE = (128, 8, critic_in_channels)
IMG_SHAPE = (128, 320, critic_in_channels)

In [ ]:
# data preprocessing
raw_files_addr = "DEAP_Dataset/data_preprocessed_python/"
transformed_files_addr = "DEAP_Dataset/transformed_data/"
file_names = os.listdir(raw_files_addr)

def trial_file_processor(file_addr, trial):
    
    channel_stft_size = (128, 8)
    start_points_list = list()
    number_of_windows = 3
    window_length = 10
    start_points_list = [[3, 3+window_length, 3+2*window_length],
                        [33, 33+window_length, 33+2*window_length]]

    with open(file_addr, 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    max_after_normalize = 1
    min_after_normalize = 0 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(len(start_points_list),channel_stft_size[0], 40*channel_stft_size[1], 3))
    
    start_point_number = 0
    for start_points in start_points_list:
        for color_channel in range(0, 3):
            for channel in range(0, 40):
                stft = librosa.stft(subject_file["data"][trial, channel,
                                              128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                              n_fft=256, hop_length=125)
                stft_db = librosa.amplitude_to_db(abs(stft))
                norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
                norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize
                norm_stft = cv2.resize(norm_stft, (channel_stft_size[1], channel_stft_size[0]))
                if color_channel_stft is None:
                    color_channel_stft = norm_stft
                else:
                    color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)
                    
            full_norm_stft[start_point_number, : , :, color_channel] = color_channel_stft
            color_channel_stft = None
            
        start_point_number = start_point_number + 1
        
    return full_norm_stft, np.array(subject_file["labels"][trial])


for file_name in file_names:
    subject_name = file_name.split(".")[0]
    for trial in range(0, 40):
        data, labels = trial_file_processor(raw_files_addr+file_name, trial)
        for sample in range(0, data.shape[0]):
            data_label_dict = {"data":data[sample], "labels":labels}
            with open(transformed_files_addr+subject_name+"t"+str(trial)+"s"+str(sample), 'wb') as handle:
                pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
transformed_files_addr = "DEAP_Dataset/transformed_data/"
generated_files_addr = "generated_data/singleGenModel/"
with open(transformed_files_addr+"s01t4s0", 'rb') as handle:
    test = pickle.load(handle)

with open(generated_files_addr+"gen_10", 'rb') as handle:
    gen = pickle.load(handle)
    

#plt.imshow(test["data"])
print(type(test["data"]))
print(test["labels"])
plt.imshow(gen["data"])
print(type(gen["data"]))
print(gen["labels"])

In [ ]:
# data generator
transformed_files_addr = "DEAP_Dataset/transformed_data/"

def train_test_spliter(addr, train_ratio, val_ratio):
    
    files = os.listdir(addr)
    number_of_files = len(files)
    random.shuffle(files)
    end_of_train_files = math.floor(train_ratio*number_of_files)
    train_files = files[0:end_of_train_files]
    validation_files = files[end_of_train_files:end_of_train_files+math.floor(val_ratio*number_of_files)]
    test_files = files[end_of_train_files+math.floor(val_ratio*number_of_files):]
    
    return train_files, validation_files, test_files
    

class DEAPDataGenerator(keras.utils.Sequence):
    
    def __init__(self, files_addrs, feature_files_addr, batch_size=32):
        self.files_addrs = files_addrs
        self.feature_files_addr = feature_files_addr
        self.batch_size = batch_size
        
        
    def number_of_samples(self):
        return len(self.files_addrs)


    def __len__(self):
        return int(np.ceil(len(self.files_addrs) / self.batch_size))

    def __getitem__(self, index):
        batch_files_addrs = self.files_addrs[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_data, batch_labels = self.__data_generation(batch_files_addrs)

        return batch_data, batch_labels

    

    def __data_generation(self, batch_file_addrs):
        batch_data = list()
        batch_labels = list()

        for batch_file_addr in batch_file_addrs:
            with open(self.feature_files_addr+batch_file_addr, 'rb') as handle:
                loaded_data = pickle.load(handle)
                batch_data.append((loaded_data["data"].astype("float32")*2.0)-1.0)
                labels = np.reshape(np.round(loaded_data["labels"]), (4, 1))
                labels = keras.utils.to_categorical(labels, 10)
                batch_labels.append(labels.astype("float32"))

        return (np.array(batch_data), np.array(batch_labels)) 
    

train_ratio = 0.8
val_ratio = 0.1
train_files, validation_files, test_files = train_test_spliter(transformed_files_addr, train_ratio, val_ratio)

train_gen = DEAPDataGenerator(train_files, transformed_files_addr)
validation_gen = DEAPDataGenerator(validation_files, transformed_files_addr)
test_gen = DEAPDataGenerator(test_files, transformed_files_addr)



In [ ]:
# copy train files
src_address = "DEAP_Dataset/transformed_data/"
dst_address = "generated_data/singleGenModel/"
#dst_address = "E:/Hamavar/proposal/generated_data/randomNoiseGenModel/"
for file_name in train_files:
    shutil.copyfile(src_address+file_name, dst_address+file_name)


In [ ]:
# model definition

def conv_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5,
):
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_partial_critic_model(input_layer):
    img_input = input_layer
    
    x = conv_block(
        img_input,
        64,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        use_bias=True,
        activation=layers.LeakyReLU(0.2),
        use_dropout=False,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        128,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        256,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        512,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )


    return x

def get_critic_model(number_of_partial_critics):
    partial_critics_input_list = list()
    partial_critics_concatted = None
    
    critic_input = layers.Input(shape=IMG_SHAPE)
    
    partial_critics_input_list = tf.split(critic_input, num_or_size_splits=number_of_partial_critics, axis=2)
    
    for critic_index in range(number_of_partial_critics):
        partial_critic_input = partial_critics_input_list[critic_index]
        if partial_critics_concatted is None:
            partial_critics_concatted = get_partial_critic_model(partial_critic_input)
        else:
            partial_critics_concatted = tf.keras.layers.Concatenate()([partial_critics_concatted,
                                                                       get_partial_critic_model(partial_critic_input)])
     
    x = conv_block(
        partial_critics_concatted,
        512,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )   
    
    
    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1)(x)
    
    c_model = keras.models.Model(critic_input, x, name="partial_critic")
    return c_model
    

def upsample_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    up_size=(2, 2),
    padding="same",
    use_bn=False,
    use_bias=True,
    use_dropout=False,
    drop_value=0.3,
):
    x = layers.UpSampling2D(up_size)(x)
    x = layers.Conv2DTranspose(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)

    if use_bn:
        x = layers.BatchNormalization()(x)

    if activation:
        x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_partial_generator_model(input_layer):
    
    x = upsample_block(
        input_layer,
        128,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x,
        128,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x, 128, layers.LeakyReLU(0.2), strides=(1, 1), padding="same", use_bias=False, use_bn=True, use_dropout=False
    )
    
    x = upsample_block(
        x, 128, layers.LeakyReLU(0.2), strides=(1, 1), padding="same", use_bias=False, use_bn=True, use_dropout=False
    )

    return x

def get_generator_model(number_of_partial_generators):
    partial_generators_output_list = list()
    
    noise = layers.Input(shape=(generator_in_channels,))
    x = layers.Dense(16 * 40 * 256, use_bias=False)(noise)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Reshape((16, 40, 256))(x)
    for generator_index in range(number_of_partial_generators):
        partial_generators_output_list.append(get_partial_generator_model(x))
    
    concatted_layer = tf.keras.layers.Concatenate(axis=3)(partial_generators_output_list)
    output_layer = layers.Conv2D(3, (2, 2), strides=(1, 1), padding="same", activation=layers.Activation("tanh"))(concatted_layer)
    
    g_model = keras.models.Model(noise, output_layer, name="generator")
    return g_model

In [ ]:
def get_critic_model_T():
    partial_critics_input_list = list()
    partial_critics_concatted = None
    critic_input = layers.Input(shape=IMG_SHAPE)
    
    critic = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same")(critic_input)
    critic = tf.keras.layers.LeakyReLU(alpha=0.2)(critic)
    
    critic = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same")(critic)
    critic = tf.keras.layers.LeakyReLU(alpha=0.2)(critic)
    
    critic = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same")(critic)
    critic = tf.keras.layers.LeakyReLU(alpha=0.2)(critic)
    
    critic = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same")(critic)
    critic = tf.keras.layers.LeakyReLU(alpha=0.2)(critic)
    
    critic = tf.keras.layers.Flatten()(critic)
    critic = tf.keras.layers.Dropout(0.4)(critic)
    critic = tf.keras.layers.Dense(1)(critic)
    
    c_model = keras.models.Model(critic_input, critic)
    
    return c_model


def get_generator_model_T():
    
    noise = layers.Input(shape=(generator_in_channels,))
    x = layers.Dense(4 * 10 * 512, use_bias=False)(noise)
    
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Reshape((4, 10, 512))(x)
    
    gen = tf.keras.layers.Conv2DTranspose(512, kernel_size=(4, 4), strides=(2, 2), padding="same")(x)
    gen = tf.keras.layers.LeakyReLU(alpha=0.2)(gen)
    
    gen = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4, 4), strides=(2, 2), padding="same")(gen)
    gen = tf.keras.layers.LeakyReLU(alpha=0.2)(gen)
    
    gen = tf.keras.layers.Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding="same")(gen)
    gen = tf.keras.layers.LeakyReLU(alpha=0.2)(gen)
    
    gen = tf.keras.layers.Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding="same")(gen)
    gen = tf.keras.layers.LeakyReLU(alpha=0.2)(gen)
    
    gen = tf.keras.layers.Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding="same")(gen)
    gen = tf.keras.layers.LeakyReLU(alpha=0.2)(gen)
    
    output_layer = tf.keras.layers.Conv2D(3, kernel_size=(7, 7), activation='tanh', padding="same")(gen)
    
    g_model = keras.models.Model(noise, output_layer, name="generator")
    
    return g_model


critic = get_critic_model_T()
generator = get_generator_model_T()


In [10]:
generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 168)]             0         
                                                                 
 dense_3 (Dense)             (None, 20480)             3440640   
                                                                 
 batch_normalization_1 (Batc  (None, 20480)            81920     
 hNormalization)                                                 
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 20480)             0         
                                                                 
 reshape_1 (Reshape)         (None, 4, 10, 512)        0         
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 8, 20, 512)       4194816   
 ranspose)                                               

In [ ]:
class ConditionalWGAN(keras.Model):
    def __init__(self, critic, generator, latent_dim, critic_extra_steps=3, gp_weight=10.0):
        super(ConditionalWGAN, self).__init__()
        self.critic = critic
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.critic_loss_tracker = keras.metrics.Mean(name="critic_loss")
        self.critic_extra_steps = critic_extra_steps
        self.gp_weight = gp_weight

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.critic_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, critic_loss_fn, generator_loss_fn):
        super(ConditionalWGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.critic_loss_fn = critic_loss_fn
        self.generator_loss_fn = generator_loss_fn
        
    def gradient_penalty(self, batch_size, real_images, fake_images):
        """Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.critic(interpolated)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp
    

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data
        #print(real_images.shape)
        #print(one_hot_labels.shape)
        #(None, 28, 56, 1)
        #(None, 2, 10)
        #Before
        #(None, 28, 28, 1)
        #(None, 10)
        one_hot_labels = tf.cast(one_hot_labels, tf.float32)

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the critic.
        valence_one_hot_labels = one_hot_labels[:, 0, :, None, None] # adds two extra dimension at the end
        valence_one_hot_labels = tf.repeat(
            valence_one_hot_labels, repeats=[image_height * image_width]
        )
        valence_one_hot_labels = tf.reshape(
            valence_one_hot_labels, (-1, image_height, image_width, num_classes_of_each_cat)
        ) # (none, 28, 28, 10)
        
        arousal_one_hot_labels = one_hot_labels[:, 1, :, None, None] # adds two extra dimension at the end
        arousal_one_hot_labels = tf.repeat(
            arousal_one_hot_labels, repeats=[image_height * image_width]
        )
        
        arousal_one_hot_labels = tf.reshape(
            arousal_one_hot_labels, (-1, image_height, image_width, num_classes_of_each_cat)
        ) # (none, 28, 28, 10)
        
        dominance_one_hot_labels = one_hot_labels[:, 2, :, None, None] # adds two extra dimension at the end
        dominance_one_hot_labels = tf.repeat(
            dominance_one_hot_labels, repeats=[image_height * image_width]
        )
        dominance_one_hot_labels = tf.reshape(
            dominance_one_hot_labels, (-1, image_height, image_width, num_classes_of_each_cat)
        ) # (none, 28, 28, 10)
        
        liking_one_hot_labels = one_hot_labels[:, 3, :, None, None] # adds two extra dimension at the end
        liking_one_hot_labels = tf.repeat(
            liking_one_hot_labels, repeats=[image_height * image_width]
        )
        liking_one_hot_labels = tf.reshape(
            liking_one_hot_labels, (-1, image_height, image_width, num_classes_of_each_cat)
        ) # (none, 28, 28, 10)
        
        image_one_hot_labels = tf.concat(
            [valence_one_hot_labels, arousal_one_hot_labels, dominance_one_hot_labels, liking_one_hot_labels], axis=-1
        )
        

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, tf.reshape(one_hot_labels, (-1, num_classes))], axis=1
        )
        

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the critic.
        for i in range(self.critic_extra_steps):
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            random_vector_labels = tf.concat(
                [random_latent_vectors, tf.reshape(one_hot_labels, (-1, num_classes))], axis=1
            )
            generated_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
            combined_images = tf.concat(
                [fake_image_and_labels, real_image_and_labels], axis=0
            )
            with tf.GradientTape() as tape:
                predictions = self.critic(combined_images)
                
                gp = self.gradient_penalty(batch_size, real_image_and_labels, fake_image_and_labels)
                d_cost = self.critic_loss_fn(labels, predictions)
                d_loss = d_cost + gp * self.gp_weight
            grads = tape.gradient(d_loss, self.critic.trainable_weights)
            self.d_optimizer.apply_gradients(
                zip(grads, self.critic.trainable_weights)
            )

            
        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, tf.reshape(one_hot_labels, (-1, num_classes))], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the critic)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            fake_images_predictions = self.critic(fake_image_and_labels)
            g_loss = self.generator_loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.critic_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.critic_loss_tracker.result(),
        }
    
    
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=4, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        saved_images = list()
        
        for i in range(self.num_img):
            
            random_latent_vectors = tf.random.normal(shape=(1, self.latent_dim))
            fake_labels = [random.randint(0,9), random.randint(0,9), random.randint(0,9), random.randint(0,9)]
            fake_labels = keras.utils.to_categorical(fake_labels, 10)
            random_vector_labels = tf.concat(
                [random_latent_vectors, tf.reshape(fake_labels, (-1, num_classes))], axis=1
            )
        
            generated_images = self.model.generator(random_vector_labels)
            generated_images = (generated_images + 1.0) / 2.0
            
            saved_images.append(generated_images[0])
        

        for i in range(self.num_img):
            plt.subplot(2, 2, i+1)
            plt.axis("off")
            plt.imshow(saved_images[i])
        
        plt.savefig(f"generated_img_{epoch}.png")
        plt.close()

In [ ]:
def critic_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


#Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)

cond_gan = ConditionalWGAN(
    critic=critic, generator=generator, latent_dim=latent_dim, critic_extra_steps=3)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9),
    critic_loss_fn = critic_loss,
    generator_loss_fn=generator_loss, #keras.losses.BinaryCrossentropy(from_logits=True)
)

cbk = GANMonitor(num_img=4, latent_dim=latent_dim)

cond_gan.fit(train_gen, epochs=1000, callbacks=[cbk])

In [ ]:
generator = keras.models.load_model('savedModels/singleGenModel.h5')
generator.summary()

In [ ]:
random_latent_vectors = tf.random.normal(shape=(1, 128))
#fake_labels = np.round([2 ,2 ,2 ,5])
fake_labels = [random.random()*9 ,random.random()*9 ,random.random()*9 ,random.random()*9]
fake_labels = np.round(fake_labels)
print(fake_labels)
fake_labels = keras.utils.to_categorical(fake_labels, 10)
random_vector_labels = tf.concat(
        [random_latent_vectors, tf.reshape(fake_labels, (-1, num_classes))], axis=1
    )
        
generated_images = generator(random_vector_labels)
generated_images = (generated_images + 1.0) / 2.0
plt.imshow(generated_images[0])
print(generated_images[0].shape)
print(fake_labels.shape)

In [ ]:
# Generate Fake Data
generated_data_files = "generated_data/singleGenModel/" 

number_of_generated_data = 3*len(train_files) 
for index in range(number_of_generated_data):
    fake_labels = [random.random()*9 ,random.random()*9 ,random.random()*9 ,random.random()*9]
    fake_labels = np.round(fake_labels, decimals=2)
    fake_labels_cat = np.round(fake_labels)
    fake_labels_cat = keras.utils.to_categorical(fake_labels, 10)
    random_latent_vectors = tf.random.normal(shape=(1, 128))
    random_vector_labels = tf.concat(
        [random_latent_vectors, tf.reshape(fake_labels_cat, (-1, num_classes))], axis=1
    )
    generated_images = generator(random_vector_labels)
    generated_images = np.array(generated_images)
    generated_images = (generated_images + 1.0) / 2.0
    
    
    data_label_dict = {"data":generated_images[0], "labels":fake_labels}
    with open(generated_data_files+"gen_"+str(index), 'wb') as handle:
        pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


In [ ]:
# Generate Fake data by random noise
generated_data_files = "generated_data/randomNoiseGenModel/" 
raw_files_addr = "DEAP_Dataset/data_preprocessed_python/"
file_names = train_files

number_of_generated_data = 3*len(train_files) 

def add_noise_to_trials(files_addr, file_name):
    
    channel_stft_size = (128, 8)
    start_points_list = list()
    number_of_windows = 3
    window_length = 10
    offset = 0
    
    
    
    subject_number = file_name[0:3]
    sample_number = file_name[-1]
    trial_number = int(file_name[-3:-2])
    
    with open(files_addr+subject_number+".dat", 'rb') as file:
        subject_file = pickle.load(file, encoding='latin1')    
    
    subject_data = subject_file["data"][trial_number]
    subject_label = subject_file["labels"][trial_number]
    
    
    if sample_number == 0:
        offset = 3
    elif sample_number == 1:
        offset = 33
    
    start_points_list = [[offset, offset+window_length, offset+2*window_length]]
    
    
    noise = np.random.normal(0,1,subject_data.shape[0]*subject_data.shape[1])
    noise = noise.reshape(subject_data.shape[0], subject_data.shape[1])
    
    subject_data = subject_data + noise
    
    max_after_normalize = 1
    min_after_normalize = 0 

    color_channel_stft = None
    full_norm_stft = np.zeros(shape=(len(start_points_list),channel_stft_size[0], 40*channel_stft_size[1], 3))
    
    start_point_number = 0
    for start_points in start_points_list:
        for color_channel in range(0, 3):
            for channel in range(0, 40):
                stft = librosa.stft(subject_data[channel,
                                              128*(start_points[color_channel]):128*(start_points[color_channel]+window_length)],
                                              n_fft=256, hop_length=125)
                stft_db = librosa.amplitude_to_db(abs(stft))
                norm_stft = (stft_db - stft_db.min())/(stft_db.max() - stft_db.min())
                norm_stft = norm_stft*(max_after_normalize - min_after_normalize) + min_after_normalize
                norm_stft = cv2.resize(norm_stft, (channel_stft_size[1], channel_stft_size[0]))
                if color_channel_stft is None:
                    color_channel_stft = norm_stft
                else:
                    color_channel_stft = np.append(color_channel_stft, norm_stft, axis=1)
                    
            full_norm_stft[start_point_number, : , :, color_channel] = color_channel_stft
            color_channel_stft = None
            
        start_point_number = start_point_number + 1
        
    return full_norm_stft, np.array(subject_label)


for index in range(number_of_generated_data):
    file_name = file_names[np.random.randint(0, len(file_names))]
    data, labels = add_noise_to_trials(raw_files_addr, file_name)
    for sample in range(0, data.shape[0]):
        data_label_dict = {"data":data[sample], "labels":labels}
        with open(generated_data_files+"noiseAdded"+"_"+str(index), 'wb') as handle:
            pickle.dump(data_label_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# estimator model data generator
estimator_train_files_addr = "generated_data/randomNoiseGenModel/"

estimator_train_files = os.listdir(estimator_train_files_addr)
random.shuffle(train_files)

class DEAPDataGenerator(keras.utils.Sequence):
    
    def __init__(self, files_addrs, feature_files_addr, batch_size=32):
        self.files_addrs = files_addrs
        self.feature_files_addr = feature_files_addr
        self.batch_size = batch_size
        
        
    def number_of_samples(self):
        return len(self.files_addrs)


    def __len__(self):
        return int(np.ceil(len(self.files_addrs) / self.batch_size))

    def __getitem__(self, index):
        batch_files_addrs = self.files_addrs[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_data, batch_labels = self.__data_generation(batch_files_addrs)

        return batch_data, batch_labels

    

    def __data_generation(self, batch_file_addrs):
        batch_data = list()
        batch_labels = list()

        for batch_file_addr in batch_file_addrs:
            with open(self.feature_files_addr+batch_file_addr, 'rb') as handle:
                loaded_data = pickle.load(handle)
                batch_data.append((loaded_data["data"].astype("float32")*2.0)-1.0)
                batch_labels.append(loaded_data["labels"])

        return (np.array(batch_data), np.array(batch_labels)) 
    


estimator_train_gen = DEAPDataGenerator(estimator_train_files, estimator_train_files_addr)
estimator_validation_gen = DEAPDataGenerator(validation_files, transformed_files_addr)
estimator_test_gen = DEAPDataGenerator(test_files, transformed_files_addr)


In [ ]:
def get_estimator_model():
    input_layer = keras.Input(shape=(128, 320, 3))

    x = tf.keras.layers.Conv2D(16, kernel_size=(5, 11), padding="same", activation="relu")(input_layer)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(rate=0.2)(x)

    x = tf.keras.layers.Conv2D(8, kernel_size=(3, 3), padding="same",activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(rate=0.2)(x)

    x = tf.keras.layers.Conv2D(4, kernel_size=(3, 3), padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Dropout(rate=0.2)(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(rate=0.1)(x)

    x = tf.keras.layers.Dense(units=512, activation="relu")(x)
    x = tf.keras.layers.Dropout(rate=0.1)(x)
    x = tf.keras.layers.Dense(units=256, activation="relu")(x)
    x = tf.keras.layers.Dropout(rate=0.1)(x)
    x = tf.keras.layers.Dense(units=128, activation="relu")(x)

    x = tf.keras.layers.Dense(units=4)(x)

    estimator_model = tf.keras.models.Model(input_layer, x)
    estimator_model.compile(loss='mae', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005)) # tf.keras.optimizers.Adam(learning_rate=0.007)

    return estimator_model

estimator_model = get_estimator_model()
estimator_model.summary()

In [ ]:
train_hist = estimator_model.fit(estimator_train_gen, validation_data=estimator_validation_gen, epochs=600)

In [ ]:
def predict_labels(Model, generator):
    predicted_labels = np.zeros(shape=(generator.number_of_samples(), 4))
    true_labels = np.zeros(shape=(generator.number_of_samples(), 4))

    counter = 0
    for data, labels in generator:
        predicted = Model.predict(data)
    
        predicted_labels[counter*batch_size:(counter+1)*batch_size, :] = predicted
        true_labels[counter*batch_size:(counter+1)*batch_size, :] = labels
        counter = counter + 1
        
    return true_labels, predicted_labels


def accuracy_evaluation_percentage(true_labels, predicted_labels):
    labels_dimension = 4
    false_predictions = np.zeros(shape=(1, labels_dimension))
    total_numbers_of_test_samples = len(predicted_labels)
    
    for prd_idx in range(0, total_numbers_of_test_samples):
        for label_idx in range(0, labels_dimension):
            if ((predicted_labels[prd_idx, label_idx]>5) and (true_labels[prd_idx, label_idx]<5)) or \
            ((predicted_labels[prd_idx, label_idx]<5) and (true_labels[prd_idx, label_idx]>5)):
                false_predictions[0, label_idx] = false_predictions[0, label_idx] + 1

    return ((total_numbers_of_test_samples - false_predictions)/total_numbers_of_test_samples)*100


In [ ]:
plt.plot(train_hist.history['loss'])
plt.plot(train_hist.history['val_loss'])

true_labels, predicted_labels = predict_labels(estimator_model, estimator_test_gen)
accuracy_evaluation = accuracy_evaluation_percentage(true_labels, predicted_labels)
print(accuracy_evaluation)

In [ ]:
generated_data_files = "E:/Hamavar/proposal/generated_data/singleGenModel/" 
train_files = os.listdir(generated_data_files)
random.shuffle(train_files)
estimator_train_gen = DEAPDataGenerator(train_files, generated_data_files)
estimator_model = get_estimator_model()

In [ ]:
train_hist = estimator_model.fit(estimator_train_gen, validation_data=estimator_validation_gen, epochs=600)

In [ ]:
plt.plot(train_hist.history['loss'])
plt.plot(train_hist.history['val_loss'])
true_labels, predicted_labels = predict_labels(estimator_model, test_gen)
accuracy_evaluation = accuracy_evaluation_percentage(true_labels, predicted_labels)
print(accuracy_evaluation)

In [ ]:
generated_data_files = "E:/Hamavar/proposal/generated_data/randomNoiseGenModel/" 
train_files = os.listdir(generated_data_files)
random.shuffle(train_files)
train_gen = DEAPDataGenerator(train_files, generated_data_files)
estimator_model = get_estimator_model()

In [ ]:
train_hist = estimator_model.fit(train_gen, validation_data=validation_gen, epochs=600)

In [ ]:
plt.plot(train_hist.history['loss'])
plt.plot(train_hist.history['val_loss'])
true_labels, predicted_labels = predict_labels(estimator_model, test_gen)
accuracy_evaluation = accuracy_evaluation_percentage(true_labels, predicted_labels)
print(accuracy_evaluation)